In [20]:
import h5py
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from filters import filter,filter_by_pix_num,cal_pix_threshold
%matplotlib inline

from utils import mvsecLoadRectificationMaps, mvsecRectifyEvents

H = 260
W = 346
GAMMA = 0.8  # 最大加权衰减率
data_path = '/home/tlab4/zjg/data/mvsec/indoor_flying1_data.hdf5'
gt_path = '/home/tlab4/zjg/data/mvsec/indoor_flying1_gt.hdf5'
data = h5py.File(data_path, 'r')
gt = h5py.File(gt_path, 'r')

left_data = data['davis']['left']
left_gt = gt['davis']['left']
print(left_data.keys())
print(left_gt.keys())


<KeysViewHDF5 ['events', 'image_raw', 'image_raw_event_inds', 'image_raw_ts', 'imu', 'imu_ts']>
<KeysViewHDF5 ['blended_image_rect', 'blended_image_rect_ts', 'depth_image_raw', 'depth_image_raw_ts', 'depth_image_rect', 'depth_image_rect_ts', 'flow_dist', 'flow_dist_ts', 'odometry', 'odometry_ts', 'pose', 'pose_ts']>


In [21]:
print(len(left_data['image_raw_event_inds']))
print(len(left_data['image_raw']))

print(len(left_gt['depth_image_rect']))
print(len(left_gt['depth_image_rect_ts']))
# print(left)

2206
2206
1399
1399


In [22]:
# load the left and right events; rectified depth map and timestamps
# 在操作list或者读取一定数据时，一定先转化成np array，能加快处理速度
e = data['davis']['left']['events'][353]
print(e)

Levents = np.array(data['davis']['left']['events'])
Revents = np.array(data['davis']['right']['events'])
print(Levents.shape, Revents.shape)

depth_ts = np.array(gt['davis']['left']['depth_image_rect_ts'])
depth_rect = np.array(gt['davis']['left']['depth_image_rect'])
print(depth_ts.shape, depth_rect.shape)

[2.49000000e+02 4.00000000e+00 1.50464518e+09 1.00000000e+00]
(14071304, 4) (12005558, 4)
(1399,) (1399, 260, 346)


In [23]:
# rectify the input left and right events
datapath = '/home/tlab4/zjg/data/mvsec'
Lx_path = datapath+'/indoor_flying_left_x_map.txt'
Ly_path = datapath+'/indoor_flying_left_y_map.txt'
Rx_path = datapath+'/indoor_flying_right_x_map.txt'
Ry_path = datapath+'/indoor_flying_right_y_map.txt'
Lx_map, Ly_map, Rx_map, Ry_map = mvsecLoadRectificationMaps(Lx_path, Ly_path, Rx_path, Ry_path)
rect_Levents = np.array(mvsecRectifyEvents(Levents, Lx_map, Ly_map)) 
rect_Revents = np.array(mvsecRectifyEvents(Revents, Rx_map, Ry_map))

print(rect_Levents.shape, rect_Revents.shape)
# 能不能加速啊？太慢了  --- 存为np.array形式
# 整流操作会把一部分的events滤除吗？有影响吗 -- 没什么太大影响



loading rectification maps...

rectifying spike coordinates...


100%|██████████| 14071304/14071304 [00:11<00:00, 1208036.53it/s]



rectifying spike coordinates...


100%|██████████| 12005558/12005558 [00:09<00:00, 1232511.32it/s]


(12004149, 4) (10041242, 4)


In [24]:
# 查看同步性
print(depth_ts[0])
print(rect_Levents[0][2])

ts1 = depth_ts[0]
ts2 = depth_ts[1]

ts_event = rect_Levents[0][2]
sum_ = 0
_sum = 0
for ts0 in depth_ts:
    ts = rect_Levents[sum_][2]
    count = 0
    while ts < ts0:
        count += 1
        ts = rect_Levents[count+sum_-1][2]
    sum_ += count
    _sum += 1
    print(count)
print(_sum)


1504645177.7113209
1504645177.4254112
1166
212
211
212
196
204
198
217
186
221
207
204
213
194
211
212
215
207
191
196
226
211
226
226
260
426
1467
2611
2603
1939
1079
509
334
447
456
418
285
356
263
217
374
879
509
670
973
1007
1500
2390
2048
3795
4628
3369
4792
4145
3830
4047
3424
4503
2969
4976
6014
6030
4613
9399
10281
10342
7358
8515
11097
19110
20449
21998
24548
28160
26361
27327
24054
23390
22811
21329
18565
16764
16482
15433
13639
12475
11749
10487
10346
9961
11195
10102
9727
10310
10383
9339
8390
8253
7807
7698
8055
8167
7519
7920
7594
9322
10348
11029
10531
8155
8534
8999
10058
9888
9395
9379
8388
9656
10419
11515
11253
11219
12211
13984
15456
16175
17464
15733
12400
9844
10223
10721
10248
11426
11237
13817
13993
14764
14860
16626
15701
13389
11403
10852
9314
9191
8435
7595
9114
9147
10078
9740
8209
7656
7929
8563
6218
5830
5589
4995
5647
8397
10429
10427
9494
8565
9123
9183
10375
9742
8908
6434
6747
8157
9360
10059
9323
9104
8035
9531
10225
10225
9965
12040
14336
19240
19685

#### 后面都用Left Event做例子

In [25]:
# 以depth_ts为基准遍历并生成event_stack，
# 每个stack都存一个对应当前depth_ts之前的events window
event_stacks = []  # 初始化events_stacks
ts_event = rect_Levents[0][2] # 先读取第一个event的timestamp
ts_sum = rect_Levents[0][2] # 
sum_event = 0  # 记录已遍历的event数量
for ts in tqdm(depth_ts):
    event = rect_Levents[sum_event]
    ts_event = event[2]

    event_window = []
    count = 0  # 记录当前depth_ts之前遍历的event数目
    while ts_event < ts:  # 遍历该depth_ts之前的所有events
        # idx = sum_event + count
        event_window.append(event)
        count += 1
        event = rect_Levents[sum_event+count]
        ts_event = event[2]
    sum_event += count

    event_stacks.append(np.array(event_window))

event_stacks = np.array(event_stacks)
print(event_stacks.shape, event_stacks[0].shape)

# 现在得到了event_stacks，其shape为：depth_ts.shape, 每个堆栈的event数量，每个event的x,y,t,p信息）

100%|██████████| 1399/1399 [00:04<00:00, 339.71it/s]

(1399,) (1165, 4)


In [26]:
def denoise(event_window_iterator):
    length = 10
    stride = 5

    real_event_stream = [] 
    noise_event_num = [] 
    noise_event_pix_num = []
    noise_event_pix_pol = []

    index_of_event_window = -1   
    for event_window in event_window_iterator:
        index_of_event_window += 1
        print('==================The index of event_window:',index_of_event_window + 1,'======================')
        real_event_stream.append(event_window) 
        # print('event_window.shape:',event_window.shape)

        flag_arr = np.zeros((H,W),dtype = int)
        pix_arr = {} 

        i = 0
        for event in event_window:
            x = int(event[1])
            y = int(event[0])
            flag_arr[x][y] = 1

            if (x,y) not in pix_arr:
                pix_arr[(x,y)] = []
            pix_arr[(x,y)].append(i)
            i += 1
                    
        index_X = range(0, W + 1 - length, stride)
        index_Y = range(0, H + 1 - length, stride) 

        Threshold_flag_min = 20
        Threshold_flag_max = 0
        num_list = []   
        flag_sum = [] 

        for index_y in index_Y:
            for index_x in index_X:
                num = 0    
                for i in range(index_y,index_y + length): 
                    for j in range(index_x,index_x + length):
                        if flag_arr[i][j] == 1:
                            num += 1
                num_list.append(num)
                flag_sum.append([(index_x,index_y),num])


        num_list.sort(reverse = True)
        while 0 in num_list:
            num_list.remove(0)
        index_min = int(len(num_list) * 0.9) - 1
        if Threshold_flag_min < num_list[index_min]:  
            Threshold_flag_min = num_list[index_min]    
        Threshold_flag_max = math.ceil(length * length * 0.9) 
        print('The candidate threshold of sparse/dense noise :',Threshold_flag_min,'-',Threshold_flag_max)
        Pix_Threshold = cal_pix_threshold(pix_arr)
        
        print('start filetering.')
        filter(Threshold_flag_min,Threshold_flag_max,flag_sum, pix_arr, Pix_Threshold, index_of_event_window, length, real_event_stream, noise_event_pix_num, noise_event_pix_pol, noise_event_num)
        print('filtering done.')

    print('The number of filtered events:',sum(noise_event_num))
    return np.array(real_event_stream)

In [27]:
def get_ratio(event_stack):  # 返回正负极性的event在该stack中的比例
    lens = len(event_stack)
    p_count = 0
    for event in event_stack:
        if event[3] == 1:
            p_count += 1  # counting the positive polarity event
    n_count = lens - p_count
    return p_count/lens, n_count/lens

In [28]:
STEP = 1

# 定义一个 激发堆栈1个subframes（channel）的events数量阈值
# 其值为 输入event数量/对应depth map的数量*2
th = int(len(rect_Levents) / len(depth_ts))  # 14808
# frames = np.ones((len(depth_ts), H, W, 3)) * 128
frames = np.zeros((len(depth_ts), H, W, 3))

# test denoise------------------------------
# event_window_iterator = np.copy(event_stacks)
# outputs = denoise(event_window_iterator)
# print(event_window_iterator.shape, outputs.shape)
#-------------------------------------------
# for i, stack in enumerate(event_stacks):
for i, stack in enumerate(event_stacks):
    print('Running on {} stack'.format(i))

    # p_ratio, n_ratio = get_ratio(stack)   # 得到正负极性事件在其中的占比

    num_channel = int(len(stack) / th) + 1 # 定义该stack需要多少channel来做加权和
    avg_event_num = int(len(stack) / num_channel) + 1  # 将event均分成num_channel这么多份

    intensity = np.zeros((num_channel, 2, H, W))  # 记录在多通道上的强度值 -- 不分极性了！！
    # intensity = np.ones((num_channel, H, W)) * 128  # 将多通道的帧初始化为强度128的帧
    
    ts = np.zeros((num_channel, 1))  # 记录每个channel结束的timestamp
    # ts[-1] = stack[-1][2]  # 先记录最后一个，最重要的timestamp
    ts_start = stack[0][2]
    ts_end = stack[-1][2]  # 最后一个timestamp
    ts_range = ts_end - ts_start
    
    channel_sum = np.zeros((2, H, W)) # 初始化几个channel加权和的通道frames
    
    idx = 0 # 记录当前stack中遍历的第几个event
    for j in range(num_channel):
        # print('the channel of {}'.format(j+1))

        freq_count = np.zeros((2, H, W)) # 记录事件在某个位置激发的频率 --- 分极性
        # count_th = 0
        while idx < ((j+1)*avg_event_num):  # 每隔一个avg的量，就累积一个frames
            x = int(stack[idx][0])
            y = int(stack[idx][1])
            pol = int(stack[idx][3])
            if pol == 1:   # 分正负极性，统计事件出现频率，并根据频率堆叠强度（2**n）
                freq = freq_count[0][y][x]
                # intensity[j][0][y][x] += (STEP + freq/8)
                intensity[j][0][y][x] += STEP
                freq_count[0][y][x] += 1
            else:
                freq = freq_count[1][y][x]
                # intensity[j][1][y][x] += (STEP + freq/8)
                intensity[j][1][y][x] += STEP
                freq_count[1][y][x] += 1

            idx += 1
            if stack[idx-1][2] == ts_end:  # 到达当前stack的最后一个timestamp，结束最后channel计数
                break
        # print('the idx of {} channel is {}'.format(j, idx))

        ts[j] = stack[idx-1][2]  # 记录每个channel结束的timestamp
        ratio_ts = (ts[j]-ts_start) / ts_range
        channel_sum += (intensity[j] * (GAMMA + (1-GAMMA)*ratio_ts))
        print('max:{}'.format(channel_sum.max()))
        print('min:{}'.format(channel_sum.min()))

        # print('the channel sum in {} channel is:{}'.format(j, channel_sum))
    frames[i,:,:,0] = channel_sum[0]
    frames[i,:,:,2] = channel_sum[1]


Running on 0 stack
max:25.0
min:0.0
Running on 1 stack
max:5.0
min:0.0
Running on 2 stack
max:5.0
min:0.0
Running on 3 stack
max:5.0
min:0.0
Running on 4 stack
max:5.0
min:0.0
Running on 5 stack
max:4.0
min:0.0
Running on 6 stack
max:4.0
min:0.0
Running on 7 stack
max:4.0
min:0.0
Running on 8 stack
max:4.0
min:0.0
Running on 9 stack
max:5.0
min:0.0
Running on 10 stack
max:4.0
min:0.0
Running on 11 stack
max:4.0
min:0.0
Running on 12 stack
max:4.0
min:0.0
Running on 13 stack
max:5.0
min:0.0
Running on 14 stack
max:4.0
min:0.0
Running on 15 stack
max:4.0
min:0.0
Running on 16 stack
max:5.0
min:0.0
Running on 17 stack
max:5.0
min:0.0
Running on 18 stack
max:5.0
min:0.0
Running on 19 stack
max:4.0
min:0.0
Running on 20 stack
max:5.0
min:0.0
Running on 21 stack
max:5.0
min:0.0
Running on 22 stack
max:5.0
min:0.0
Running on 23 stack
max:4.0
min:0.0
Running on 24 stack
max:5.0
min:0.0
Running on 25 stack
max:5.0
min:0.0
Running on 26 stack
max:5.0
min:0.0
Running on 27 stack
max:5.0
min:0.0
R

In [32]:
frames_cp =frames[:]
frames_mapped = []
root = '/home/tlab4/cj/data/data_denoise'
for i in range(len(frames_cp)):
    frame = frames_cp[i]
    value_range = frame.max() - frame.min()
    frame_map = (frame / value_range) * 255 
    frames_mapped.append(frame_map)
    # frame_map = np.interp(frame, (frame.min(), frame.max()), (0, 255))
    # cv2.imwrite(root+'/{}.png'.format(i), frame_map).
# frames_mapped = np.clip(frames, 0, 255)

In [36]:
frames_mapped = np.array(frames_mapped)
for f in frames_mapped:
    print(f.min())
    print(f.max())

0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0
0.0
255.0


In [31]:
root = '/home/tlab4/cj/data_fixed'
b = np.zeros((480, 480, 3))
b[:,:, 0] = np.ones((480, 480)) * 128

cv2.imwrite(root+'/1.jpg', b)

False

In [14]:
import numpy as np

arr1 = np.zeros((1211, 260, 346, 3))
arr2 = np.ones((1391, 260, 346, 3))

arr_concat = np.concatenate((arr1, arr2), axis=0)
k = list(arr_concat)
print(arr_concat)  # (2602, 260, 346, 3)


[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0

In [10]:
import numpy as np
a = [[1,2],[4,7]]
b = [[242,452],[3,46],[4,6],[7,4]]
e = [[4,5], [14,9],[7,9],[0,1]]
c = np.array(a)
d = np.array(b)
f = np.concatenate((c,d,e), axis=0)
f

array([[  1,   2],
       [  4,   7],
       [242, 452],
       [  3,  46],
       [  4,   6],
       [  7,   4],
       [  4,   5],
       [ 14,   9],
       [  7,   9],
       [  0,   1]])

In [18]:
import h5py
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils import mvsecLoadRectificationMaps, mvsecRectifyEvents
from tools import load_mvsec, get_event_stacks, denoise, FrameGenerator, write_to_file

H = 260
W = 346
GAMMA = 0.8 # largest decay ratio

# def main():
framesL = []
framesR = []
depth = []

for i in range(1, 5):
    # load data and ground truth
    datapath = '/home/tlab4/zjg/data/mvsec/indoor_flying' + str(i) + '_data.hdf5'
    gtpath = '/home/tlab4/zjg/data/mvsec/indoor_flying' + str(i) + '_gt.hdf5'
    rect_Levents, rect_Revents, depth_rect, depth_ts = load_mvsec(datapath, gtpath)
    # get the left and right event stacks
    Levent_stacks = get_event_stacks(depth_ts, rect_Levents)
    Revent_stacks = get_event_stacks(depth_ts, rect_Revents)
    # get the frames
    Lframes = FrameGenerator(rect_Levents, depth_ts, Levent_stacks)
    Rframes = FrameGenerator(rect_Revents, depth_ts, Revent_stacks)

    framesL.append(Lframes)
    framesR.append(Rframes)
    depth.append(depth_rect)

# if __name__ == '__main__':
#     main()



loading rectification maps...

rectifying spike coordinates...


100%|██████████| 14071304/14071304 [00:11<00:00, 1251224.80it/s]



rectifying spike coordinates...


100%|██████████| 12005558/12005558 [00:09<00:00, 1268046.28it/s]


Generating event stacks......


...

Generating event stacks......


100%|██████████| 390/390 [00:02<00:00, 191.29it/s] 


Done with the event stacks!
Generating event stacks......


100%|██████████| 390/390 [00:01<00:00, 260.59it/s] 


Done with the event stacks!
Accumulating events to frames......
Done with generating the frames!
Accumulating events to frames......
Done with generating the frames!


In [27]:
framesL[2].max()

22.0